<a href="https://colab.research.google.com/github/MuhTawfik/tensor_models/blob/main/classification_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [ ]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import pandas as pd


Dataset 

In [ ]:
CSV_COUMN_NAMES=['SepalLength','SpalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginica']

In [ ]:
train_path=tf.keras.utils.get_file('iris_training.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path=tf.keras.utils.get_file('iris_test.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train=pd.read_csv(train_path,names=CSV_COUMN_NAMES, header=0)
test=pd.read_csv(test_path,names=CSV_COUMN_NAMES, header=0)

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [ ]:
train.head()

,SepalLength,SpalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y=train.pop('Species')
test_y=test.pop('Species')

In [ ]:
train.shape

(120, 4)

input function 


In [ ]:
def input_fn(features,labels,training=True,batch_size=256):
  #convert input to dataset
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
        #shaffle and repeat if you are in training mode
        dataset=dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

feature columns

In [ ]:
my_feature_columns=[]

for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SpalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


build the model
- DNN classifier (deep neural network)
- LinearClassifier

In [ ]:
#build a DNN with 2 hidden layers with 30 and 10 hidden nodes each 
classifier=tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #two hodden layer of 30 and 10 nodes respectibely
    hidden_units=[30,10],
    #the model must choose between 3 classes.
    n_classes=3

)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7p27pkqa', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(
    input_fn=lambda:input_fn(train,train_y,training=True),
 steps=5000 #as epoch
)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp7p27pkqa/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.5801326, step = 0
INFO:tensorflow:global_step/sec: 365.351
INFO:tensorflow:loss = 1.2876277, step = 100 (0.279 sec)
INFO:tensorflow:global_step/sec: 448.962
INFO:tensorflow:loss = 1.170839, step = 200 (0.223 sec)
INFO:tensorflow:global_step/sec:

In [ ]:
eval_result=classifier.evaluate(input_fn=lambda: input_fn(test,test_y,training=False))
print('\nTest set Accuracy: {accuracy:0.2f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-11T09:13:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7p27pkqa/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.25134s
INFO:tensorflow:Finished evaluation at 2022-01-11-09:13:36
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss = 0.6241022, global_step = 5000, loss = 0.6241022
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp7p27pkqa/model.ckpt-5000

Test set Accuracy: 0.70



In [ ]:

def input_fn(features, batch_size=256):
  # Convert the inputs to a Dataset without labels.
  return tf.data.Dataset.from_tensor_slices(dict (features)).batch(batch_size)
#'SepalLength','SpalWidth','PetalLength','PetalWidth','Species
features = ['SepalLength','SpalWidth','PetalLength','PetalWidth']
predict={} 

print ("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input (feature + ": ")
    if not val.isdigit(): valid = False
  predict[feature] = [float(val)]
predictions = classifier.predict (input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  print ('Prediction is "{}" ({:.1f}%)' . format (SPECIES[class_id], 100 * probability))


Please type numeric values as prompted.
SepalLength: 152.2
SpalWidth: 15.2
PetalLength: 2.2
PetalWidth: 3.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7p27pkqa/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Versicolor" (100.0%)


In [ ]:
train[train.SepalLength>1
      ]

,SepalLength,SpalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3
...,...,...,...,...
115,5.5,2.6,4.4,1.2
116,5.7,3.0,4.2,1.2
117,4.4,2.9,1.4,0.2
118,4.8,3.0,1.4,0.1
